In [1]:
import os
import pandas as pd
import numpy as np
import time, gc
import cv2
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pretrainedmodels
from argparse import Namespace
from sklearn.utils import shuffle
from apex import amp
from sklearn.model_selection import StratifiedKFold
from efficientnet_pytorch import EfficientNet
from cvcore.data.auto_augment import RandAugment
from PIL import Image
from utils import bn_update, moving_average, copy_model


In [2]:
!ls /home/chec/data/bengali

class_map.csv		       train.csv
sample_submission.csv	       train.csv.zip
test.csv		       train_image_data_0.parquet
test_image_data_0.parquet      train_image_data_0.parquet.zip
test_image_data_0.parquet.zip  train_image_data_1.parquet
test_image_data_1.parquet      train_image_data_1.parquet.zip
test_image_data_1.parquet.zip  train_image_data_2.parquet
test_image_data_2.parquet      train_image_data_2.parquet.zip
test_image_data_2.parquet.zip  train_image_data_3.parquet
test_image_data_3.parquet      train_image_data_3.parquet.zip
test_image_data_3.parquet.zip


In [3]:
#!ls /home/chec/data/bengali

In [4]:
DATA_DIR = '/home/chec/data/bengali'

In [5]:
train_df = pd.read_csv(f'{DATA_DIR}/train.csv')
test_df = pd.read_csv(f'{DATA_DIR}/test.csv')
class_map_df = pd.read_csv(f'{DATA_DIR}/class_map.csv')
sample_sub_df = pd.read_csv(f'{DATA_DIR}/sample_submission.csv')

In [6]:
train_df.head()

,image_id,grapheme_root,vowel_diacritic,consonant_diacritic,grapheme
0,Train_0,15,9,5,ক্ট্রো
1,Train_1,159,0,0,হ
2,Train_2,22,3,5,খ্রী
3,Train_3,53,2,2,র্টি
4,Train_4,71,9,5,থ্রো


In [7]:
HEIGHT = 137
WIDTH = 236

In [8]:
#import albumentations as albu
def get_train_augs():
    return RandAugment(n=2, m=27)

In [9]:
#plt.imshow(x)

In [10]:
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms


class BengaliDataset(Dataset):
    def __init__(self, df, img_df, train_mode=True, test_mode=False):
        self.df = df
        self.img_df = img_df
        self.train_mode = train_mode
        self.test_mode = test_mode

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = self.get_img(row.image_id)
        orig_img = img.copy()
        #print(img.shape)
        if self.train_mode:
            augs = get_train_augs()
            #img = augs(image=img)['image']
            img = np.asarray(augs(Image.fromarray(img)))
        
        img = np.expand_dims(img, axis=-1)
        orig_img = np.expand_dims(orig_img, axis=-1)
        
        #print('###', img.shape)
        #img = np.concatenate([img, img, img], 2)
        #print('>>>', img.shape)
        
        # taken from https://www.kaggle.com/iafoss/image-preprocessing-128x128
        #MEAN = [ 0.06922848809290576,  0.06922848809290576,  0.06922848809290576]
        #STD = [ 0.20515700083327537,  0.20515700083327537,  0.20515700083327537]
        
        img = transforms.functional.to_tensor(img)
        orig_img = transforms.functional.to_tensor(orig_img)
        
        #img = transforms.functional.normalize(img, mean=MEAN, std=STD)
        
        if self.test_mode:
            return img
        elif self.train_mode:
            return img, orig_img, torch.tensor([row.grapheme_root, row.vowel_diacritic, row.consonant_diacritic, row.word_label])
        else:
            return img, torch.tensor([row.grapheme_root, row.vowel_diacritic, row.consonant_diacritic, row.word_label])
                    
    def get_img(self, img_id):
        return 255 - self.img_df.loc[img_id].values.reshape(HEIGHT, WIDTH).astype(np.uint8)

    def __len__(self):
        return len(self.df)
    
def get_train_val_loaders(batch_size=4, val_batch_size=4, ifold=0, dev_mode=False):
    train_df = pd.read_csv(f'{DATA_DIR}/train.csv')

    train_df = shuffle(train_df, random_state=1234)

    grapheme_words = np.unique(train_df.grapheme.values)
    grapheme_words_dict = {grapheme: i for i, grapheme in enumerate(grapheme_words)}
    train_df['word_label'] = train_df['grapheme'].map(lambda x: grapheme_words_dict[x])

    print(train_df.shape)

    if dev_mode:
        img_df = pd.read_parquet(f'{DATA_DIR}/train_image_data_0.parquet').set_index('image_id')
        train_df = train_df.iloc[:1000]
    else:
        img_dfs = [pd.read_parquet(f'{DATA_DIR}/train_image_data_{i}.parquet') for i in range(4)]
        img_df = pd.concat(img_dfs, axis=0).set_index('image_id')
    print(img_df.shape)
    #split_index = int(len(train_df) * 0.9)
    
    #train = train_df.iloc[:split_index]
    #val = train_df.iloc[split_index:]
    
    kf = StratifiedKFold(5, random_state=1234, shuffle=True)
    for i, (train_idx, val_idx) in enumerate(kf.split(train_df, train_df['grapheme_root'].values)):
        if i == ifold:
            #print(val_idx)
            train = train_df.iloc[train_idx]
            val = train_df.iloc[val_idx]
            break
    assert i == ifold
    print(train.shape, val.shape)
    
    train_ds = BengaliDataset(train, img_df, True, False)
    val_ds = BengaliDataset(val, img_df, False, False)
    
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=8, drop_last=True)
    train_loader.num = len(train_ds)

    val_loader = DataLoader(val_ds, batch_size=val_batch_size, shuffle=False, num_workers=8, drop_last=False)
    val_loader.num = len(val_ds)

    return train_loader, val_loader

In [11]:
#train_loader, val_loader = get_train_val_loaders()

In [12]:
#for x in train_loader:
#    print(x)
#    break

# model

In [13]:
#import pretrainedmodels

In [14]:
#model_name = 'resnet50' # could be fbresnet152 or inceptionresnetv2
#model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet').cuda()
#model.eval()

In [15]:
from argparse import Namespace
import timm
from timm.models.activations import Swish, Mish
from timm.models.adaptive_avgmax_pool import SelectAdaptivePool2d

MEAN = [ 0.06922848809290576 ]
STD = [ 0.20515700083327537 ]

class BengaliNet(nn.Module):
    """
    EfficientNet B0-B8.
    Args:
        cfg (CfgNode): configs
    """
    def __init__(self, cfg):
        super(BengaliNet, self).__init__()
        model_name = cfg.MODEL_NAME
        pretrained = cfg.PRETRAINED
        input_channels = cfg.IN_CHANNELS
        pool_type = cfg.POOL_TYPE
        drop_connect_rate = cfg.DROP_CONNECT
        self.drop_rate = cfg.DROPOUT
        cls_head = cfg.CLS_HEAD
        num_total_classes = cfg.NUM_GRAPHEME_CLASSES + cfg.NUM_VOWEL_CLASSES + cfg.NUM_CONSONANT_CLASSES \
            + cfg.NUM_WORD_CLASSES

        backbone = timm.create_model(
            model_name=model_name,
            pretrained=pretrained,
            in_chans=input_channels,
            drop_connect_rate=drop_connect_rate,
        )
        self.conv_stem = backbone.conv_stem
        self.bn1 = backbone.bn1
        self.act1 = backbone.act1
        ### Original blocks ###
        for i in range(len((backbone.blocks))):
            setattr(self, "block{}".format(str(i)), backbone.blocks[i])
        self.conv_head = backbone.conv_head
        self.bn2 = backbone.bn2
        self.act2 = backbone.act2
        self.aux_block5 = backbone.blocks[5]
        self.aux_num_features = self.block5[-1].bn3.num_features
        self.aux_head4 = nn.Conv2d(self.aux_num_features, self.aux_num_features * 4, kernel_size=(1, 1), stride=(1, 1), bias=False)
        self.bn4 = nn.BatchNorm2d(self.aux_num_features * 4)
        self.act4 = Swish()
        self.aux_head5 = nn.Conv2d(self.aux_num_features, self.aux_num_features * 4, kernel_size=(1, 1), stride=(1, 1), bias=False)
        self.bn5 = nn.BatchNorm2d(self.aux_num_features * 4)
        self.act5 = Swish()
        self.global_pool = SelectAdaptivePool2d(pool_type=pool_type)
        self.num_features = backbone.num_features * self.global_pool.feat_mult()
        assert cls_head == 'linear'
        if cls_head == "linear":
            ### Baseline head ###
            self.fc = nn.Linear(self.num_features, num_total_classes)            
            self.aux_fc1 = nn.Linear(self.aux_num_features*4, num_total_classes)
            self.aux_fc2 = nn.Linear(self.aux_num_features*4, num_total_classes)
            
            for fc in [self.fc, self.aux_fc1, self.aux_fc2]:
                nn.init.zeros_(fc.bias.data)
        elif cls_head == "norm_softmax":
            ### NormSoftmax ###
            self.grapheme_fc = NormSoftmax(self.num_features, num_grapheme_classes)
            self.consonant_fc = NormSoftmax(self.num_features, num_consonant_classes)
            self.vowel_fc = NormSoftmax(self.num_features, num_vowel_classes)
        # Replace with Mish activation
        if cfg.MODEL_ACTIVATION == "mish":
            convert_swish_to_mish(self)
        del backbone

    def _features(self, x):
        x = self.conv_stem(x)
        x = self.bn1(x)
        x = self.act1(x)
        x = self.block0(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x); b4 = x
        x = self.block5(x); b4 = self.aux_block5(b4); b5 = x
        x = self.block6(x)
        x = self.conv_head(x); b4 = self.aux_head4(b4); b5 = self.aux_head5(b5)
        x = self.bn2(x); b4 = self.bn4(b4); b5 = self.bn5(b5)
        x = self.act2(x); b4 = self.act4(b4); b5 = self.act5(b5)
        return b4, b5, x

    def forward(self, x):
        x = F.interpolate(x, size=(380, 380), mode='bilinear', align_corners=False)
        for i in range(len(x)):
            transforms.functional.normalize(x[i], mean=MEAN, std=STD, inplace=True)

        # _, _, x = self._features(x)
        b4, b5, x = self._features(x)
        x = self.global_pool(x); b4 = self.global_pool(b4); b5 = self.global_pool(b5)
        x = torch.flatten(x, 1); b4 = torch.flatten(b4, 1); b5 = torch.flatten(b5, 1)
        if self.drop_rate > 0.:
            x = F.dropout(x, p=self.drop_rate, training=self.training)
        logits = self.fc(x)
        
        aux_logits1 = self.aux_fc1(b4)
        aux_logits2 = self.aux_fc2(b5)
        
        return logits, aux_logits1, aux_logits2

In [16]:
MODEL_DIR = './model4-ckps'
def create_model(cfg):
    model = BengaliNet(cfg)
    model_file = os.path.join(MODEL_DIR, cfg.MODEL_NAME, cfg.CKP_NAME)

    parent_dir = os.path.dirname(model_file)
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)

    print('model file: {}, exist: {}'.format(model_file, os.path.exists(model_file)))

    if os.path.exists(model_file):
        print('loading {}...'.format(model_file))
        model.load_state_dict(torch.load(model_file))
    
    return model, model_file

In [17]:
#bnet = BengaliNet('se_resnext50_32x4d').cuda()

In [18]:
#bnet(torch.randn((2, 1, 137, 236)).cuda()).size()

# train

In [19]:
round(1/9, 6)

0.111111

In [20]:
import numpy as np
import sklearn.metrics
import torch


def calc_metrics(preds0, preds1, preds2, preds3, y):
    assert len(y) == len(preds0) == len(preds1) == len(preds2) == len(preds3)

    recall_grapheme = sklearn.metrics.recall_score(preds0, y[:, 0], average='macro')
    recall_vowel = sklearn.metrics.recall_score(preds1, y[:, 1], average='macro')
    recall_consonant = sklearn.metrics.recall_score(preds2, y[:, 2], average='macro')
    recall_word = sklearn.metrics.recall_score(preds3, y[:, 3], average='macro')
    
    scores = [recall_grapheme, recall_vowel, recall_consonant]
    final_recall_score = np.average(scores, weights=[2, 1, 1])
    
    metrics = {}
    metrics['recall'] = round(final_recall_score, 6)
    metrics['recall_grapheme'] = round(recall_grapheme, 6)
    metrics['recall_vowel'] = round(recall_vowel, 6)
    metrics['recall_consonant'] = round(recall_consonant, 6)
    metrics['recall_word'] = round(recall_word, 6)
    
    metrics['acc_grapheme'] = round((preds0 == y[:, 0]).sum() / len(y), 6)
    metrics['acc_vowel'] = round((preds1 == y[:, 1]).sum() / len(y), 6)
    metrics['acc_consonant'] = round((preds2 == y[:, 2]).sum() / len(y), 6)
    metrics['acc_word'] = round((preds3 == y[:, 3]).sum() / len(y), 6)    
    
    return metrics

In [21]:
def criterion(outputs, y_true):
    # outputs: (N, 182)
    # y_true: (N, 3)
    
    outputs = torch.split(outputs, [168, 11, 7, 1295], dim=1)
    loss0 = F.cross_entropy(outputs[0], y_true[:, 0], reduction='mean')
    loss1 = F.cross_entropy(outputs[1], y_true[:, 1], reduction='mean')
    loss2 = F.cross_entropy(outputs[2], y_true[:, 2], reduction='mean')
    loss3 = F.cross_entropy(outputs[3], y_true[:, 3], reduction='mean')
    
    return loss0 + loss1 + loss2 + loss3 #, loss0.item(), loss1.item(), loss2.item()

In [22]:
def validate(model, val_loader):
    model.eval()
    loss0, loss1, loss2, loss3 = 0., 0., 0., 0.
    preds0, preds1, preds2, preds3 = [], [], [], []
    y_true = []
    with torch.no_grad():
        for x, y in val_loader:
            y_true.append(y)
            x, y = x.cuda(), y.cuda()
            outputs, _, _ = model(x)
            outputs = torch.split(outputs, [168, 11, 7, 1295], dim=1)
            
            preds0.append(torch.max(outputs[0], dim=1)[1])
            preds1.append(torch.max(outputs[1], dim=1)[1])
            preds2.append(torch.max(outputs[2], dim=1)[1])
            preds3.append(torch.max(outputs[3], dim=1)[1])
            loss0 += F.cross_entropy(outputs[0], y[:, 0], reduction='sum').item()
            loss1 += F.cross_entropy(outputs[1], y[:, 1], reduction='sum').item()
            loss2 += F.cross_entropy(outputs[2], y[:, 2], reduction='sum').item()
            loss3 += F.cross_entropy(outputs[3], y[:, 3], reduction='sum').item()
            
            # for debug
            #metrics = {}
            #metrics['loss_grapheme'] =  F.cross_entropy(outputs[0], y[:, 0], reduction='mean').item()
            #metrics['loss_vowel'] =  F.cross_entropy(outputs[1], y[:, 1], reduction='mean').item()
            #metrics['loss_consonant'] =  F.cross_entropy(outputs[2], y[:, 2], reduction='mean').item()
            #return metrics
    
    preds0 = torch.cat(preds0, 0).cpu().numpy()
    preds1 = torch.cat(preds1, 0).cpu().numpy()
    preds2 = torch.cat(preds2, 0).cpu().numpy()
    preds3 = torch.cat(preds3, 0).cpu().numpy()
    
    y_true = torch.cat(y_true, 0).numpy()
    
    #print('y_true:', y_true.shape)
    #print('preds0:', preds0.shape)
    
    metrics = calc_metrics(preds0, preds1, preds2, preds3, y_true)
    metrics['loss_grapheme'] = round(loss0 / val_loader.num, 6)
    metrics['loss_vowel'] = round(loss1 / val_loader.num, 6)
    metrics['loss_consonant'] = round(loss2 / val_loader.num, 6)
    metrics['loss_word'] = round(loss3 / val_loader.num, 6)
    
    return metrics
            

In [23]:
def get_lrs(optimizer):
    lrs = []
    for pgs in optimizer.state_dict()['param_groups']:
        lrs.append(pgs['lr'])
    lrs = ['{:.6f}'.format(x) for x in lrs]
    return lrs

In [24]:
def save_model(model, model_file):
    parent_dir = os.path.dirname(model_file)
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)
    if isinstance(model, nn.DataParallel):
        torch.save(model.module.state_dict(), model_file)
    else:
        torch.save(model.state_dict(), model_file)

In [25]:
def mixup(data, targets, alpha=1):
    indices = torch.randperm(data.size(0))
    shuffled_data = data[indices]
    shuffled_targets = targets[indices]

    lam = np.random.beta(alpha, alpha)
    data = data * lam + shuffled_data * (1 - lam)
    targets = (targets, shuffled_targets, lam)

    return data, targets


def mixup_criterion(outputs, targets):
    targets1, targets2, lam = targets
    #criterion = nn.CrossEntropyLoss(reduction='mean')
    return lam * criterion(outputs, targets1) + (1 - lam) * criterion(outputs, targets2)

In [26]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [27]:
def rand_bbox_new(size, lam):
    H = size[2]
    W = size[3]

    x_margin_rate = 0.2

    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * (1-x_margin_rate*2) * cut_rat)
    cut_h = np.int(H * cut_rat)
    
    min_x_center = np.int(W * x_margin_rate + cut_w / 2)
    max_x_center = np.int(W * (1-x_margin_rate) - cut_w / 2)
    #print(min_x_center, max_x_center, lam, cut_w)
    min_y_center = cut_h // 2
    max_y_center = H - cut_h // 2
    if max_y_center == min_y_center:
        max_y_center += 1

    # uniform
    cx = np.random.randint(min_x_center, max_x_center)
    cy = np.random.randint(min_y_center, max_y_center)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)
    
    #print(bbx1, bbx2, bby1, bby2)

    return bbx1, bby1, bbx2, bby2

In [28]:
np.random.random()

0.029572411820954048

In [29]:
from over9000.over9000 import Over9000
from over9000.radam import RAdam
from gridmask import GridMask

In [30]:
from cvcore.solver import WarmupCyclicalLR
def make_optimizer(model, base_lr=4e-4, weight_decay=0., weight_decay_bias=0., epsilon=1e-3):
    """
    Create optimizer with per-layer learning rate and weight decay.
    """
    params = []
    for key, value in model.named_parameters():
        if not value.requires_grad:
            continue
        lr = base_lr
        params += [{"params": [value], "lr": lr, "weight_decay": weight_decay_bias if 'bias' in key else weight_decay}]
    
    optimizer = torch.optim.AdamW(params, lr, eps=epsilon)
    return optimizer

In [31]:
import time
def train_epoch(args, model, train_loader, epoch, optimizer, lr_scheduler, grid):
    train_loss = 0
    bg = time.time()

    for batch_idx, (img, orig_img, targets) in enumerate(train_loader):
        img, orig_img, targets  = img.cuda(), orig_img.cuda(), targets.cuda()
        batch_size = img.size(0)
        r = np.random.rand()

        if r < 0.3:
            # generate mixed sample
            lam = np.random.beta(args.beta, args.beta)
            rand_index = torch.randperm(img.size()[0]).cuda()
            target_a = targets
            target_b = targets[rand_index]
            bbx1, bby1, bbx2, bby2 = rand_bbox(img.size(), lam)
            #img[:, :, bby1:bby2, bbx1:bbx2] = img[rand_index, :, bby1:bby2, bbx1:bbx2] #for new cutmix
            img[:, :, bbx1:bbx2, bby1:bby2] = img[rand_index, :, bbx1:bbx2, bby1:bby2]
            
            # adjust lambda to exactly match pixel ratio
            lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (img.size()[-1] * img.size()[-2]))
            # compute output
            outputs, outputs_aux1, outputs_aux2 = model(img)
            loss_primary = criterion(outputs, target_a) * lam + criterion(outputs, target_b) * (1. - lam)
            loss_aux1 = criterion(outputs_aux1, target_a) * lam + criterion(outputs_aux1, target_b) * (1. - lam)
            loss_aux2 = criterion(outputs_aux2, target_a) * lam + criterion(outputs_aux2, target_b) * (1. - lam)
            loss = loss_primary + (loss_aux1 + loss_aux2)*0.8
        elif r > 0.7:
            img = grid(img)
            outputs, outputs_aux1, outputs_aux2 = model(img)
            loss_primary = criterion(outputs, targets)
            loss_aux1 = criterion(outputs_aux1, targets)
            loss_aux2 = criterion(outputs_aux2, targets)
            loss = loss_primary + (loss_aux1 + loss_aux2)*0.8
        else:
            orig_img, targets = mixup(orig_img, targets)
            outputs, outputs_aux1, outputs_aux2 = model(orig_img)
            loss_primary = mixup_criterion(outputs, targets)
            loss_aux1 = mixup_criterion(outputs_aux1, targets)
            loss_aux2 = mixup_criterion(outputs_aux2, targets)
            loss = loss_primary + (loss_aux1 + loss_aux2)*0.8
            #loss = criterion(outputs, targets)

        optimizer.zero_grad()
        with amp.scale_loss(loss, optimizer) as scaled_loss:
            scaled_loss.backward()
        #loss.backward()
        lr_scheduler(optimizer, batch_idx, epoch)
        optimizer.step()            
        
        current_lr = get_lrs(optimizer)

        train_loss += loss.item()
        print('\r {:4d} | {:.6f} | {:06d}/{} | {:.4f} | {:.4f} | {:.2f} |'.format(
            epoch, float(current_lr[0]), batch_size*(batch_idx+1), train_loader.num, 
            loss.item(), train_loss/(batch_idx+1), (time.time() - bg) / 60), end='')


In [32]:
import copy
best_metrics = 0.
best_metrics_swa = 0.


def validate_and_save(model, model_file, val_loader, save=False):
    global best_metrics
    best_key = 'recall'
    val_metrics = validate(model, val_loader)
    print('\nval:', val_metrics)
    
    if val_metrics[best_key] > best_metrics:
        best_metrics = val_metrics[best_key]
        if save:
            save_model(model, model_file)
            print('###>>>>> saved', model_file)
    model.train()

def validate_and_save_swa(model, model_file, val_loader, save=False):
    global best_metrics_swa
    best_key = 'recall'
    val_metrics = validate(model, val_loader)
    print('\nval:', val_metrics)
    
    if val_metrics[best_key] > best_metrics_swa:
        best_metrics_swa = val_metrics[best_key]
        if save:
            save_model(model, model_file)
            print('###>>>>> saved', model_file)
    model.train()


def train(args):
    model, model_file = create_model(cfg)
    model = model.cuda()

    swa_cfg = copy.deepcopy(cfg)
    swa_cfg.CKP_NAME = cfg.CKP_NAME + '_swa'
    swa_model, swa_model_file = create_model(swa_cfg)
    swa_model = swa_model.cuda()

    optimizer = make_optimizer(model)
    lr_scheduler = WarmupCyclicalLR(
        "cos", args.base_lr, args.num_epochs, iters_per_epoch=len(train_loader), warmup_epochs=args.warmup_epochs)
    
    [model, swa_model], optimizer = amp.initialize([model, swa_model], optimizer, opt_level="O1",verbosity=0)
    
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
        swa_model = nn.DataParallel(swa_model)
    
    validate_and_save(model, model_file, val_loader, save=False)

    swa_model_loaded = False
    if os.path.exists(swa_model_file):
        swa_model_loaded = True
        validate_and_save_swa(swa_model, swa_model_file, val_loader, save=False)
    
    for cycle in range(1, args.num_cycles+1):
        print('CYCLE:', cycle)
        grid = GridMask(64, 128, rotate=15, ratio=0.6, mode=1, prob=1.)

        for epoch in range(args.start_epoch, args.num_epochs):
            grid.set_prob(epoch, args.st_epochs)
            train_epoch(args, model, train_loader, epoch, optimizer, lr_scheduler, grid)
            validate_and_save(model, model_file, val_loader, save=True)
            
            if (epoch+1) == args.swa_start and cycle == 1:
                if not swa_model_loaded:
                    copy_model(swa_model, model)
                swa_n = args.swa_n
            if (epoch+1) >= args.swa_start and (epoch+1) % args.swa_freq == 0:
                print('SWA>>>:')
                moving_average(swa_model, model, 1.0 / (swa_n + 1))
                swa_n += 1
                bn_update(train_loader, swa_model)
                validate_and_save_swa(swa_model, swa_model_file, val_loader, save=True)

        #args.base_lr = 4e-4
        #args.num_epochs = 60
        #args.warmup_epochs = 3

        # reset scheduler at each cycle
        lr_scheduler = WarmupCyclicalLR(
            "cos", args.base_lr, args.num_epochs, iters_per_epoch=len(train_loader), warmup_epochs=args.warmup_epochs)

In [33]:
cfg = Namespace()
cfg.MODEL_NAME = 'tf_efficientnet_b4'
cfg.PRETRAINED = True
cfg.IN_CHANNELS = 1
cfg.POOL_TYPE = 'avg'
cfg.CLS_HEAD = 'linear'
cfg.MODEL_ACTIVATION = 'swish'
cfg.DROP_CONNECT = 0.2
cfg.DROPOUT= 0.
cfg.NUM_WORD_CLASSES = 1295
cfg.NUM_GRAPHEME_CLASSES = 168
cfg.NUM_VOWEL_CLASSES = 11
cfg.NUM_CONSONANT_CLASSES = 7
cfg.CKP_NAME = 'model4_eb4_fold3_380.pth'

In [34]:
#model, model_file = create_model(cfg)
#model(torch.randn(2,1,137,236))[0].size()

In [35]:
args = Namespace()

args.base_lr = 3e-4
args.num_epochs = 60
args.start_epoch = 0
args.warmup_epochs = 10
args.num_cycles = 100
args.batch_size = 128
args.val_batch_size = 512
args.st_epochs = 5

args.swa_start = 2
args.swa_freq = 3
args.swa_n = 3

args.beta = 1.0
args.cutmix_prob = 0.5

In [36]:
train_loader, val_loader = get_train_val_loaders(batch_size=args.batch_size, val_batch_size=args.val_batch_size, ifold=3)

(200840, 6)
(200840, 32332)
(160716, 6) (40124, 6)


In [ ]:
train(args)

model file: ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3_380.pth, exist: True
loading ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3_380.pth...
model file: ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3_380.pth_swa, exist: True
loading ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3_380.pth_swa...

val: {'recall': 0.998467, 'recall_grapheme': 0.99772, 'recall_vowel': 0.998871, 'recall_consonant': 0.999558, 'recall_word': 0.99743, 'acc_grapheme': 0.997582, 'acc_vowel': 0.998704, 'acc_consonant': 0.999477, 'acc_word': 0.997458, 'loss_grapheme': 0.012791, 'loss_vowel': 0.006768, 'loss_consonant': 0.002529, 'loss_word': 0.01347}

val: {'recall': 0.998677, 'recall_grapheme': 0.998053, 'recall_vowel': 0.998936, 'recall_consonant': 0.999667, 'recall_word': 0.997745, 'acc_grapheme': 0.998056, 'acc_vowel': 0.998829, 'acc_consonant': 0.999477, 'acc_word': 0.997732, 'loss_grapheme': 0.011116, 'loss_vowel': 0.005994, 'loss_consonant': 0.002174, 'loss_word': 0.012167}
CYCLE: 1
    0 | 

  0%|          | 0/1255 [00:00<?, ?it/s]


val: {'recall': 0.998139, 'recall_grapheme': 0.997313, 'recall_vowel': 0.998481, 'recall_consonant': 0.999447, 'recall_word': 0.997014, 'acc_grapheme': 0.997209, 'acc_vowel': 0.99853, 'acc_consonant': 0.999227, 'acc_word': 0.997009, 'loss_grapheme': 0.01459, 'loss_vowel': 0.006974, 'loss_consonant': 0.003406, 'loss_word': 0.014635}
SWA>>>:


100%|██████████| 1255/1255 [07:07<00:00,  2.90it/s]



val: {'recall': 0.998648, 'recall_grapheme': 0.998056, 'recall_vowel': 0.99884, 'recall_consonant': 0.999639, 'recall_word': 0.997759, 'acc_grapheme': 0.998056, 'acc_vowel': 0.998829, 'acc_consonant': 0.999477, 'acc_word': 0.997732, 'loss_grapheme': 0.011008, 'loss_vowel': 0.005916, 'loss_consonant': 0.00213, 'loss_word': 0.012015}
    3 | 0.000119 | 160640/160716 | 0.0217 | 5.2222 | 14.18 ||
val: {'recall': 0.998229, 'recall_grapheme': 0.997337, 'recall_vowel': 0.998754, 'recall_consonant': 0.999489, 'recall_word': 0.997086, 'acc_grapheme': 0.997358, 'acc_vowel': 0.998579, 'acc_consonant': 0.999302, 'acc_word': 0.997134, 'loss_grapheme': 0.01389, 'loss_vowel': 0.007072, 'loss_consonant': 0.003256, 'loss_word': 0.01489}
    4 | 0.000147 | 160640/160716 | 0.1496 | 5.0842 | 14.36 ||
val: {'recall': 0.997871, 'recall_grapheme': 0.996877, 'recall_vowel': 0.998279, 'recall_consonant': 0.99945, 'recall_word': 0.996458, 'acc_grapheme': 0.996685, 'acc_vowel': 0.99828, 'acc_consonant': 0.99912

...

100%|██████████| 1255/1255 [07:10<00:00,  2.80it/s]



val: {'recall': 0.998613, 'recall_grapheme': 0.997923, 'recall_vowel': 0.998899, 'recall_consonant': 0.999706, 'recall_word': 0.997718, 'acc_grapheme': 0.997857, 'acc_vowel': 0.998903, 'acc_consonant': 0.999526, 'acc_word': 0.997682, 'loss_grapheme': 0.011301, 'loss_vowel': 0.006171, 'loss_consonant': 0.002185, 'loss_word': 0.012525}
   12 | 0.000267 | 160640/160716 | 0.7925 | 5.2547 | 14.47 ||
val: {'recall': 0.997558, 'recall_grapheme': 0.996561, 'recall_vowel': 0.998552, 'recall_consonant': 0.99856, 'recall_word': 0.9962, 'acc_grapheme': 0.996536, 'acc_vowel': 0.998405, 'acc_consonant': 0.998779, 'acc_word': 0.996237, 'loss_grapheme': 0.016791, 'loss_vowel': 0.007631, 'loss_consonant': 0.004435, 'loss_word': 0.017615}
   13 | 0.000261 | 160640/160716 | 6.8214 | 5.5810 | 14.43 ||
val: {'recall': 0.997958, 'recall_grapheme': 0.997171, 'recall_vowel': 0.998488, 'recall_consonant': 0.999003, 'recall_word': 0.996637, 'acc_grapheme': 0.997059, 'acc_vowel': 0.998181, 'acc_consonant': 0.99

  0%|          | 0/1255 [00:00<?, ?it/s]


val: {'recall': 0.997666, 'recall_grapheme': 0.996767, 'recall_vowel': 0.998333, 'recall_consonant': 0.998798, 'recall_word': 0.996232, 'acc_grapheme': 0.996635, 'acc_vowel': 0.99833, 'acc_consonant': 0.998754, 'acc_word': 0.996262, 'loss_grapheme': 0.018253, 'loss_vowel': 0.009038, 'loss_consonant': 0.005094, 'loss_word': 0.018969}
SWA>>>:


 90%|████████▉ | 1126/1255 [06:25<00:43,  2.94it/s]

In [73]:
train(args)

model file: ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3_380.pth, exist: True
loading ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3_380.pth...
model file: ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3_380.pth_swa, exist: True
loading ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3_380.pth_swa...

val: {'recall': 0.998467, 'recall_grapheme': 0.99772, 'recall_vowel': 0.998871, 'recall_consonant': 0.999558, 'recall_word': 0.99743, 'acc_grapheme': 0.997582, 'acc_vowel': 0.998704, 'acc_consonant': 0.999477, 'acc_word': 0.997458, 'loss_grapheme': 0.012791, 'loss_vowel': 0.006768, 'loss_consonant': 0.002529, 'loss_word': 0.01347}

val: {'recall': 0.998671, 'recall_grapheme': 0.998067, 'recall_vowel': 0.998935, 'recall_consonant': 0.999613, 'recall_word': 0.997673, 'acc_grapheme': 0.997981, 'acc_vowel': 0.998829, 'acc_consonant': 0.999477, 'acc_word': 0.997632, 'loss_grapheme': 0.011121, 'loss_vowel': 0.006063, 'loss_consonant': 0.002289, 'loss_word': 0.012252}
CYCLE: 1
    0 | 

  0%|          | 0/1255 [00:00<?, ?it/s]


val: {'recall': 0.997929, 'recall_grapheme': 0.996827, 'recall_vowel': 0.998736, 'recall_consonant': 0.999326, 'recall_word': 0.996394, 'acc_grapheme': 0.996885, 'acc_vowel': 0.99848, 'acc_consonant': 0.999178, 'acc_word': 0.996411, 'loss_grapheme': 0.01605, 'loss_vowel': 0.007821, 'loss_consonant': 0.003694, 'loss_word': 0.017375}
SWA>>>:


100%|██████████| 1255/1255 [07:05<00:00,  2.83it/s]



val: {'recall': 0.998568, 'recall_grapheme': 0.997907, 'recall_vowel': 0.998851, 'recall_consonant': 0.999607, 'recall_word': 0.997571, 'acc_grapheme': 0.997857, 'acc_vowel': 0.998779, 'acc_consonant': 0.999452, 'acc_word': 0.997558, 'loss_grapheme': 0.011218, 'loss_vowel': 0.006183, 'loss_consonant': 0.00223, 'loss_word': 0.012368}
    3 | 0.000099 | 160640/160716 | 0.0395 | 5.0809 | 14.22 ||
val: {'recall': 0.998415, 'recall_grapheme': 0.9975, 'recall_vowel': 0.999015, 'recall_consonant': 0.999644, 'recall_word': 0.997189, 'acc_grapheme': 0.997458, 'acc_vowel': 0.998729, 'acc_consonant': 0.999477, 'acc_word': 0.997184, 'loss_grapheme': 0.014277, 'loss_vowel': 0.00677, 'loss_consonant': 0.002857, 'loss_word': 0.014951}
    4 | 0.000123 | 160640/160716 | 13.6762 | 5.1560 | 14.33 |
val: {'recall': 0.99774, 'recall_grapheme': 0.996756, 'recall_vowel': 0.998246, 'recall_consonant': 0.999204, 'recall_word': 0.996235, 'acc_grapheme': 0.99666, 'acc_vowel': 0.998455, 'acc_consonant': 0.99897

...

  0%|          | 0/1255 [00:00<?, ?it/s]


val: {'recall': 0.998031, 'recall_grapheme': 0.996973, 'recall_vowel': 0.998634, 'recall_consonant': 0.999546, 'recall_word': 0.996625, 'acc_grapheme': 0.996935, 'acc_vowel': 0.998679, 'acc_consonant': 0.999302, 'acc_word': 0.996635, 'loss_grapheme': 0.015263, 'loss_vowel': 0.007387, 'loss_consonant': 0.003292, 'loss_word': 0.016521}
SWA>>>:


100%|██████████| 1255/1255 [07:12<00:00,  2.86it/s]



val: {'recall': 0.998644, 'recall_grapheme': 0.997961, 'recall_vowel': 0.998903, 'recall_consonant': 0.999751, 'recall_word': 0.997802, 'acc_grapheme': 0.997956, 'acc_vowel': 0.998804, 'acc_consonant': 0.999601, 'acc_word': 0.997782, 'loss_grapheme': 0.011001, 'loss_vowel': 0.005979, 'loss_consonant': 0.002175, 'loss_word': 0.01192}
   45 | 0.000034 | 108544/160716 | 3.6602 | 5.2251 | 9.82 ||

KeyboardInterrupt: 

In [37]:
train(args)

model file: ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3_380.pth, exist: True
loading ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3_380.pth...
model file: ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3_380.pth_swa, exist: True
loading ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3_380.pth_swa...

val: {'recall': 0.998467, 'recall_grapheme': 0.99772, 'recall_vowel': 0.998871, 'recall_consonant': 0.999558, 'recall_word': 0.99743, 'acc_grapheme': 0.997582, 'acc_vowel': 0.998704, 'acc_consonant': 0.999477, 'acc_word': 0.997458, 'loss_grapheme': 0.012791, 'loss_vowel': 0.006768, 'loss_consonant': 0.002529, 'loss_word': 0.01347}

val: {'recall': 0.998648, 'recall_grapheme': 0.998048, 'recall_vowel': 0.998873, 'recall_consonant': 0.999623, 'recall_word': 0.997515, 'acc_grapheme': 0.997931, 'acc_vowel': 0.998704, 'acc_consonant': 0.999402, 'acc_word': 0.997483, 'loss_grapheme': 0.011498, 'loss_vowel': 0.00635, 'loss_consonant': 0.002381, 'loss_word': 0.012641}
CYCLE: 1
   14 | 0

  0%|          | 0/1255 [00:00<?, ?it/s]


val: {'recall': 0.997934, 'recall_grapheme': 0.997128, 'recall_vowel': 0.998158, 'recall_consonant': 0.999324, 'recall_word': 0.996713, 'acc_grapheme': 0.99686, 'acc_vowel': 0.99843, 'acc_consonant': 0.999028, 'acc_word': 0.99671, 'loss_grapheme': 0.014749, 'loss_vowel': 0.007822, 'loss_consonant': 0.003959, 'loss_word': 0.015996}
SWA>>>:


100%|██████████| 1255/1255 [07:12<00:00,  2.93it/s]



val: {'recall': 0.998589, 'recall_grapheme': 0.997942, 'recall_vowel': 0.998871, 'recall_consonant': 0.999601, 'recall_word': 0.997536, 'acc_grapheme': 0.997832, 'acc_vowel': 0.998704, 'acc_consonant': 0.999427, 'acc_word': 0.997508, 'loss_grapheme': 0.011495, 'loss_vowel': 0.006407, 'loss_consonant': 0.002351, 'loss_word': 0.012667}
   16 | 0.000222 | 160640/160716 | 5.9235 | 5.1297 | 14.50 ||
val: {'recall': 0.997779, 'recall_grapheme': 0.996753, 'recall_vowel': 0.998259, 'recall_consonant': 0.999351, 'recall_word': 0.996096, 'acc_grapheme': 0.996785, 'acc_vowel': 0.998355, 'acc_consonant': 0.999128, 'acc_word': 0.996137, 'loss_grapheme': 0.016279, 'loss_vowel': 0.008563, 'loss_consonant': 0.004194, 'loss_word': 0.017798}
   17 | 0.000214 | 160640/160716 | 5.4592 | 5.3904 | 14.49 ||

...

   39 | 0.000029 | 160640/160716 | 5.5000 | 4.9362 | 14.54 ||
val: {'recall': 0.998016, 'recall_grapheme': 0.997107, 'recall_vowel': 0.998442, 'recall_consonant': 0.999409, 'recall_word': 0.996583, 'acc_grapheme': 0.997034, 'acc_vowel': 0.99848, 'acc_consonant': 0.999302, 'acc_word': 0.996635, 'loss_grapheme': 0.015601, 'loss_vowel': 0.007675, 'loss_consonant': 0.003668, 'loss_word': 0.017253}
   40 | 0.000023 | 160640/160716 | 0.2301 | 5.0696 | 14.56 ||
val: {'recall': 0.99831, 'recall_grapheme': 0.997554, 'recall_vowel': 0.99866, 'recall_consonant': 0.999473, 'recall_word': 0.997267, 'acc_grapheme': 0.997632, 'acc_vowel': 0.998704, 'acc_consonant': 0.999327, 'acc_word': 0.997283, 'loss_grapheme': 0.012165, 'loss_vowel': 0.006354, 'loss_consonant': 0.002903, 'loss_word': 0.013325}
   41 | 0.000019 | 160640/160716 | 4.6941 | 5.0980 | 14.58 ||

  0%|          | 0/1255 [00:00<?, ?it/s]


val: {'recall': 0.997915, 'recall_grapheme': 0.996847, 'recall_vowel': 0.998599, 'recall_consonant': 0.99937, 'recall_word': 0.996576, 'acc_grapheme': 0.996935, 'acc_vowel': 0.998505, 'acc_consonant': 0.999227, 'acc_word': 0.996611, 'loss_grapheme': 0.015353, 'loss_vowel': 0.007471, 'loss_consonant': 0.00361, 'loss_word': 0.016937}
SWA>>>:


 21%|██▏       | 269/1255 [01:36<05:38,  2.91it/s]

KeyboardInterrupt: 

In [39]:
train(args)

model file: ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3_380.pth, exist: True
loading ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3_380.pth...
model file: ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3_380.pth_swa, exist: True
loading ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3_380.pth_swa...

val: {'recall': 0.998314, 'recall_grapheme': 0.997668, 'recall_vowel': 0.998427, 'recall_consonant': 0.99949, 'recall_word': 0.99722, 'acc_grapheme': 0.997333, 'acc_vowel': 0.998654, 'acc_consonant': 0.999302, 'acc_word': 0.997234, 'loss_grapheme': 0.012546, 'loss_vowel': 0.006781, 'loss_consonant': 0.002767, 'loss_word': 0.013686}

val: {'recall': 0.998499, 'recall_grapheme': 0.997926, 'recall_vowel': 0.998543, 'recall_consonant': 0.999602, 'recall_word': 0.997673, 'acc_grapheme': 0.997882, 'acc_vowel': 0.998779, 'acc_consonant': 0.999427, 'acc_word': 0.997657, 'loss_grapheme': 0.010988, 'loss_vowel': 0.006149, 'loss_consonant': 0.00255, 'loss_word': 0.011906}
CYCLE: 1
    8 | 

  0%|          | 0/1255 [00:00<?, ?it/s]


val: {'recall': 0.997023, 'recall_grapheme': 0.99578, 'recall_vowel': 0.997481, 'recall_consonant': 0.999051, 'recall_word': 0.995257, 'acc_grapheme': 0.995788, 'acc_vowel': 0.997882, 'acc_consonant': 0.998928, 'acc_word': 0.995265, 'loss_grapheme': 0.019129, 'loss_vowel': 0.010337, 'loss_consonant': 0.005143, 'loss_word': 0.020291}
SWA>>>:


100%|██████████| 1255/1255 [08:40<00:00,  2.45it/s]



val: {'recall': 0.997864, 'recall_grapheme': 0.997037, 'recall_vowel': 0.998158, 'recall_consonant': 0.999223, 'recall_word': 0.996839, 'acc_grapheme': 0.997084, 'acc_vowel': 0.998405, 'acc_consonant': 0.999103, 'acc_word': 0.99681, 'loss_grapheme': 0.013349, 'loss_vowel': 0.007275, 'loss_consonant': 0.003111, 'loss_word': 0.014714}
   10 | 0.000266 | 160640/160716 | 0.5633 | 5.5519 | 16.03 ||
val: {'recall': 0.997647, 'recall_grapheme': 0.996431, 'recall_vowel': 0.998337, 'recall_consonant': 0.999389, 'recall_word': 0.996037, 'acc_grapheme': 0.996461, 'acc_vowel': 0.99843, 'acc_consonant': 0.998978, 'acc_word': 0.996062, 'loss_grapheme': 0.016803, 'loss_vowel': 0.008233, 'loss_consonant': 0.004633, 'loss_word': 0.017464}
   11 | 0.000259 | 160640/160716 | 13.2253 | 5.4043 | 16.07 |

...

  0%|          | 0/1255 [00:00<?, ?it/s]


val: {'recall': 0.997595, 'recall_grapheme': 0.996199, 'recall_vowel': 0.998607, 'recall_consonant': 0.999375, 'recall_word': 0.996147, 'acc_grapheme': 0.996411, 'acc_vowel': 0.998405, 'acc_consonant': 0.998928, 'acc_word': 0.996137, 'loss_grapheme': 0.017194, 'loss_vowel': 0.007709, 'loss_consonant': 0.005101, 'loss_word': 0.017654}
SWA>>>:


100%|██████████| 1255/1255 [08:40<00:00,  2.29it/s]



val: {'recall': 0.998589, 'recall_grapheme': 0.997888, 'recall_vowel': 0.998875, 'recall_consonant': 0.999705, 'recall_word': 0.997811, 'acc_grapheme': 0.997857, 'acc_vowel': 0.998878, 'acc_consonant': 0.999526, 'acc_word': 0.997782, 'loss_grapheme': 0.011042, 'loss_vowel': 0.006001, 'loss_consonant': 0.002163, 'loss_word': 0.012028}
   10 | 0.000266 | 160640/160716 | 1.4577 | 5.2489 | 16.14 ||
val: {'recall': 0.997704, 'recall_grapheme': 0.996388, 'recall_vowel': 0.998795, 'recall_consonant': 0.999247, 'recall_word': 0.996381, 'acc_grapheme': 0.99671, 'acc_vowel': 0.998754, 'acc_consonant': 0.998928, 'acc_word': 0.996336, 'loss_grapheme': 0.019258, 'loss_vowel': 0.009315, 'loss_consonant': 0.006259, 'loss_word': 0.018882}
   11 | 0.000263 | 063872/160716 | 12.7251 | 5.5997 | 6.42 |

KeyboardInterrupt: 

In [ ]:
train(args)

model file: ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3_380.pth, exist: True
loading ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3_380.pth...
model file: ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3_380.pth_swa, exist: False

val: {'recall': 0.998018, 'recall_grapheme': 0.997036, 'recall_vowel': 0.998639, 'recall_consonant': 0.999361, 'recall_word': 0.996448, 'acc_grapheme': 0.99691, 'acc_vowel': 0.998629, 'acc_consonant': 0.999202, 'acc_word': 0.996461, 'loss_grapheme': 0.014569, 'loss_vowel': 0.006717, 'loss_consonant': 0.003164, 'loss_word': 0.015258}
CYCLE: 1
    0 | 0.000150 | 160640/160716 | 12.2697 | 5.3472 | 14.08 |
val: {'recall': 0.998146, 'recall_grapheme': 0.997216, 'recall_vowel': 0.998845, 'recall_consonant': 0.999307, 'recall_word': 0.996611, 'acc_grapheme': 0.996959, 'acc_vowel': 0.998579, 'acc_consonant': 0.999103, 'acc_word': 0.996685, 'loss_grapheme': 0.01638, 'loss_vowel': 0.007743, 'loss_consonant': 0.004658, 'loss_word': 0.016503}
###>>>>> saved ./m

  0%|          | 0/1255 [00:00<?, ?it/s]


val: {'recall': 0.99752, 'recall_grapheme': 0.996393, 'recall_vowel': 0.998183, 'recall_consonant': 0.99911, 'recall_word': 0.995869, 'acc_grapheme': 0.996187, 'acc_vowel': 0.99823, 'acc_consonant': 0.998878, 'acc_word': 0.995838, 'loss_grapheme': 0.021373, 'loss_vowel': 0.012239, 'loss_consonant': 0.007318, 'loss_word': 0.020206}
SWA>>>:


100%|██████████| 1255/1255 [07:33<00:00,  2.68it/s]



val: {'recall': 0.997533, 'recall_grapheme': 0.996394, 'recall_vowel': 0.998198, 'recall_consonant': 0.999147, 'recall_word': 0.996213, 'acc_grapheme': 0.996361, 'acc_vowel': 0.998255, 'acc_consonant': 0.998903, 'acc_word': 0.996112, 'loss_grapheme': 0.015758, 'loss_vowel': 0.008725, 'loss_consonant': 0.00417, 'loss_word': 0.017115}
###>>>>> saved ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3_380.pth_swa
   10 | 0.000266 | 160640/160716 | 0.7321 | 5.7232 | 14.63 ||
val: {'recall': 0.996911, 'recall_grapheme': 0.99547, 'recall_vowel': 0.998028, 'recall_consonant': 0.998674, 'recall_word': 0.995199, 'acc_grapheme': 0.995514, 'acc_vowel': 0.998081, 'acc_consonant': 0.998604, 'acc_word': 0.99509, 'loss_grapheme': 0.021331, 'loss_vowel': 0.010744, 'loss_consonant': 0.006966, 'loss_word': 0.022242}
   11 | 0.000259 | 160640/160716 | 5.5887 | 5.7066 | 14.59 ||

...


val: {'recall': 0.998427, 'recall_grapheme': 0.997885, 'recall_vowel': 0.998303, 'recall_consonant': 0.999635, 'recall_word': 0.997612, 'acc_grapheme': 0.997882, 'acc_vowel': 0.998729, 'acc_consonant': 0.999452, 'acc_word': 0.997582, 'loss_grapheme': 0.01097, 'loss_vowel': 0.006126, 'loss_consonant': 0.002464, 'loss_word': 0.01187}
   48 | 0.000000 | 160640/160716 | 2.1133 | 5.2048 | 14.68 ||
val: {'recall': 0.998108, 'recall_grapheme': 0.997342, 'recall_vowel': 0.998362, 'recall_consonant': 0.999386, 'recall_word': 0.996799, 'acc_grapheme': 0.997084, 'acc_vowel': 0.998629, 'acc_consonant': 0.999202, 'acc_word': 0.99686, 'loss_grapheme': 0.01433, 'loss_vowel': 0.007816, 'loss_consonant': 0.003238, 'loss_word': 0.015761}
   49 | 0.000000 | 160640/160716 | 13.2333 | 5.0441 | 14.73 |

  0%|          | 0/1255 [00:00<?, ?it/s]


val: {'recall': 0.997921, 'recall_grapheme': 0.99699, 'recall_vowel': 0.998526, 'recall_consonant': 0.999179, 'recall_word': 0.996649, 'acc_grapheme': 0.996835, 'acc_vowel': 0.998654, 'acc_consonant': 0.999003, 'acc_word': 0.996685, 'loss_grapheme': 0.014779, 'loss_vowel': 0.008082, 'loss_consonant': 0.003597, 'loss_word': 0.016215}
SWA>>>:


100%|██████████| 1255/1255 [07:39<00:00,  2.70it/s]



val: {'recall': 0.998449, 'recall_grapheme': 0.997936, 'recall_vowel': 0.998287, 'recall_consonant': 0.999635, 'recall_word': 0.997677, 'acc_grapheme': 0.997906, 'acc_vowel': 0.998704, 'acc_consonant': 0.999452, 'acc_word': 0.997632, 'loss_grapheme': 0.010934, 'loss_vowel': 0.006118, 'loss_consonant': 0.002438, 'loss_word': 0.011852}
CYCLE: 2
    0 | 0.000150 | 160640/160716 | 6.1674 | 5.2896 | 14.20 ||
val: {'recall': 0.997694, 'recall_grapheme': 0.996673, 'recall_vowel': 0.998375, 'recall_consonant': 0.999054, 'recall_word': 0.996153, 'acc_grapheme': 0.996635, 'acc_vowel': 0.998405, 'acc_consonant': 0.998779, 'acc_word': 0.996162, 'loss_grapheme': 0.016575, 'loss_vowel': 0.008763, 'loss_consonant': 0.004649, 'loss_word': 0.018412}
    1 | 0.000299 | 160640/160716 | 11.3982 | 5.3649 | 14.31 |
val: {'recall': 0.997435, 'recall_grapheme': 0.996553, 'recall_vowel': 0.997768, 'recall_consonant': 0.998867, 'recall_word': 0.995956, 'acc_grapheme': 0.996162, 'acc_vowel': 0.998106, 'acc_cons

In [37]:
train(args)

model file: ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3_380.pth, exist: True
loading ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3_380.pth...
model file: ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3_380.pth_swa, exist: False

val: {'recall': 0.997597, 'recall_grapheme': 0.996428, 'recall_vowel': 0.998394, 'recall_consonant': 0.999137, 'recall_word': 0.996368, 'acc_grapheme': 0.996735, 'acc_vowel': 0.99843, 'acc_consonant': 0.998953, 'acc_word': 0.996361, 'loss_grapheme': 0.018075, 'loss_vowel': 0.010609, 'loss_consonant': 0.006052, 'loss_word': 0.017806}
CYCLE: 1
    0 | 0.000200 | 160640/160716 | 6.2135 | 5.5802 | 14.02 ||
val: {'recall': 0.996769, 'recall_grapheme': 0.995271, 'recall_vowel': 0.997749, 'recall_consonant': 0.998786, 'recall_word': 0.995084, 'acc_grapheme': 0.995464, 'acc_vowel': 0.997807, 'acc_consonant': 0.998679, 'acc_word': 0.995065, 'loss_grapheme': 0.027358, 'loss_vowel': 0.016393, 'loss_consonant': 0.011132, 'loss_word': 0.02422}
    1 | 0.000199 |

KeyboardInterrupt: 

In [37]:
train(args)

model file: ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3_380.pth, exist: True
loading ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3_380.pth...
model file: ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3_380.pth_swa, exist: False

val: {'recall': 0.796988, 'recall_grapheme': 0.763845, 'recall_vowel': 0.863414, 'recall_consonant': 0.796848, 'recall_word': 0.762264, 'acc_grapheme': 0.493819, 'acc_vowel': 0.788904, 'acc_consonant': 0.767645, 'acc_word': 0.316245, 'loss_grapheme': 1.992506, 'loss_vowel': 0.626105, 'loss_consonant': 0.522644, 'loss_word': 3.290409}
CYCLE: 1


/home/chec/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


    0 | 0.000292 | 160640/160716 | 6.7174 | 10.7151 | 13.99 ||
val: {'recall': 0.992886, 'recall_grapheme': 0.989107, 'recall_vowel': 0.996141, 'recall_consonant': 0.997187, 'recall_word': 0.9882, 'acc_grapheme': 0.989607, 'acc_vowel': 0.996087, 'acc_consonant': 0.996087, 'acc_word': 0.988212, 'loss_grapheme': 0.129831, 'loss_vowel': 0.068911, 'loss_consonant': 0.05198, 'loss_word': 0.144907}
###>>>>> saved ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3_380.pth
    1 | 0.000543 | 160640/160716 | 7.1504 | 7.3232 | 14.15 ||
val: {'recall': 0.992768, 'recall_grapheme': 0.989784, 'recall_vowel': 0.995191, 'recall_consonant': 0.996311, 'recall_word': 0.989335, 'acc_grapheme': 0.99018, 'acc_vowel': 0.995364, 'acc_consonant': 0.995464, 'acc_word': 0.988884, 'loss_grapheme': 0.069323, 'loss_vowel': 0.043396, 'loss_consonant': 0.0354, 'loss_word': 0.072684}
    2 | 0.000476 | 160640/160716 | 16.1799 | 6.6101 | 14.24 |
val: {'recall': 0.994403, 'recall_grapheme': 0.992025, 'recall_vowel': 0.9

KeyboardInterrupt: 

In [44]:
time.time()

1582104595.1300395

In [ ]:
#save_model(model, model_file)

In [ ]:
del model